In [1]:
import warnings
import gc
import os
import random
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
from glob import glob
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, f1_score

warnings.filterwarnings(action = 'ignore')

In [2]:
train_2017 = []
train_2018 = []
train_2019 = []
train_2020 = []

for i, path in enumerate(sorted(glob("Data/train/2.FeatureEngineering_train*"))):
    if i == 0:
        train_2017 = pd.read_csv(path)
        pass
    elif i==1:
        train_2018 = pd.read_csv(path)
        pass
    elif i==2:
        train_2019 = pd.read_csv(path)
        pass
    else:
        train_2020 = pd.read_csv(path)
        pass
    
test_2017 = []
test_2018 = []
test_2019 = []
test_2020 = []

for i, path in enumerate(sorted(glob("Data/test/2.FeatureEngineering_test*"))):
    if i == 0:
        test_2017 = pd.read_csv(path)
        pass
    elif i==1:
        test_2018 = pd.read_csv(path)
        pass
    elif i==2:
        test_2019 = pd.read_csv(path)
        pass
    else:
        test_2020 = pd.read_csv(path)
        pass

In [3]:
sub = pd.read_csv('Data/sample_submission.csv')

In [4]:
train = [train_2017, train_2018, train_2019, train_2020]
test = [test_2017, test_2018, test_2019, test_2020]

scale = MinMaxScaler()

In [5]:
for n, (train, test) in enumerate(zip(train, test)):
    train_x = train.drop(['idx', 'knowcode'], axis = 1)
    train_y = train['knowcode']

    test_x = test.drop(['idx'], axis = 1)
    #
    train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size = 0.2, shuffle=True)

    train_x = scale.fit_transform(train_x)
   #
    valid_x = scale.transform(valid_x)
    test_x = scale.transform(test_x)
    
    RF = RandomForestClassifier(n_estimators = 4000)
    RF.fit(train_x, train_y)
    #
    pred = RF.predict(valid_x)
    print(n, '번째 정확도 :', accuracy_score(valid_y, pred))
    print(n, '번째 끝')
    if n == 0 :
        sub['knowcode'][:len(test_2017)] = RF.predict(test_x)
    elif n == 1 :
        sub['knowcode'][len(test_2017):len(test_2017) + len(test_2018)] = RF.predict(test_x)
    elif n == 2 :
        sub['knowcode'][len(test_2017) + len(test_2018):len(test_2017) + len(test_2018) + len(test_2019)] = RF.predict(test_x)    
    elif n == 3 :
        sub['knowcode'][len(test_2017) + len(test_2018) + len(test_2019) :len(test_2017) + len(test_2018) + len(test_2019) + len(test_2020)] = RF.predict(test_x)    

0 번째 정확도 : 0.5858798735511064
0 번째 끝
1 번째 정확도 : 0.6203856749311295
1 번째 끝
2 번째 정확도 : 0.5981538461538461
2 번째 끝


AttributeError: 'list' object has no attribute 'drop'

In [ ]:
sub.to_csv('Data/Final_submission.csv', index = False)